# Single Player Game Demonstration

First, we set up

In [1]:
import re
import random
from kosa import Client
client = Client()

We join a game as the green player with industrial matrix.

In [2]:
client.join_a_game(color='green', player_mat='INDUSTRIAL')
client.start()
print('game id ', client.game_id)

game id  3ffe2ec1-593a-4d29-8497-e6a0eeab59aa


We know performa mind blowing set of actions.

In [3]:
def perform_action_matching(action, regex):
    available_actions = client.get_available_actions()
    if action in available_actions:
        option_list = client.get_available_options(action)
        index_to_play = None
        for idx, val in enumerate(option_list):
            result = re.compile(regex).findall(val)
            if len(result) > 0:
                index_to_play = idx
                break
    
        if index_to_play is not None:
            client.perform_action(action, index_to_play)
    
for _ in range(16):
    perform_action_matching('MOVE', 'GainCoin')
    perform_action_matching('TRADE', 'Popularity')
    perform_action_matching('MOVE', 'GainCoin')
    perform_action_matching('BOLSTER', 'BolsterPower')
    perform_action_matching('PRODUCE', 'm1')
    
perform_action_matching('MOVE', 'GainCoin')
perform_action_matching('TRADE', 'TradePopularity')
perform_action_matching('MOVE', 'GainCoin')
perform_action_matching('TRADE', 'TradePopularity')

We have now obtained 2 stars, which we can check as follows:

In [4]:
def get_star_events():
    game = client.export_game()
    return [g for g in game if g['type'] == 'StarEvent']

def print_star_count():
    print("Number of stars " , len(get_star_events()))
    
print_star_count()

Number of stars  2


# Moving around
The following performs some moves to place workers.

In [6]:
perform_action_matching('MOVE','WORKER_2.*v1.*VILLAGE')
perform_action_matching('PRODUCE','VILLAGE')
perform_action_matching('MOVE', 'GainCoin')
perform_action_matching('PRODUCE','VILLAGE')

In [7]:
perform_action_matching('MOVE', 'WORKER_1.*f1.*FARM')
perform_action_matching('PRODUCE','VILLAGE')
perform_action_matching('MOVE', 'WORKER_1.*t2.*TUNDRA')
perform_action_matching('PRODUCE','VILLAGE')
perform_action_matching('MOVE', 'WORKER_1.*m3.*MOUNTAIN')
perform_action_matching('PRODUCE','VILLAGE')
perform_action_matching('MOVE', 'WORKER_1.*w3.*FOREST')
perform_action_matching('PRODUCE','VILLAGE')

In [8]:
get_star_events() # We now have all workers

[{'id': '167bb64e-3f67-4184-a0d4-7f0226a71397',
  'playerId': '08ce4984-b041-48a5-a316-5c084a389da3',
  'star': 'MAX_POWER',
  'type': 'StarEvent'},
 {'id': 'b4e15269-975c-4d9d-9cbd-dba865fea7dd',
  'playerId': '08ce4984-b041-48a5-a316-5c084a389da3',
  'star': 'MAX_POPULARITY',
  'type': 'StarEvent'},
 {'id': '04f92fae-8c33-46fa-ac92-6f5ec599c042',
  'playerId': '08ce4984-b041-48a5-a316-5c084a389da3',
  'star': 'ALL_WORKERS',
  'type': 'StarEvent'}]

In [9]:
perform_action_matching('MOVE', 'WORKER_7.*f1.*FARM')
perform_action_matching('PRODUCE','TUNDRA')
perform_action_matching('MOVE', 'WORKER_7.*t2.*TUNDRA')
perform_action_matching('PRODUCE','TUNDRA')
perform_action_matching('MOVE', 'WORKER_7.*m3.*MOUNYAIN')
perform_action_matching('PRODUCE','MOUNTAIN')
perform_action_matching('MOVE', 'WORKER_7.*v5.*VILLAGE')
perform_action_matching('PRODUCE','MOUNTAIN')

In [10]:
perform_action_matching('MOVE', 'WORKER_4.*f1.*FARM')
perform_action_matching('PRODUCE','FOREST')
perform_action_matching('MOVE', 'WORKER_4.*m1.*MOUNTAIN')
perform_action_matching('PRODUCE','MOUNTAIN')

In [11]:
perform_action_matching('MOVE', 'WORKER_3.*f1.*FARM')
perform_action_matching('PRODUCE','FOREST')
perform_action_matching('MOVE', 'WORKER_3.*t2.*TUNDRA')
perform_action_matching('PRODUCE','FOREST')

In [12]:
perform_action_matching('MOVE', 'WORKER_8.*f1.*FARM')
perform_action_matching('PRODUCE','FOREST')
perform_action_matching('MOVE', 'WORKER_8.*t2.*TUNDRA')
perform_action_matching('PRODUCE','TUNDRA')
perform_action_matching('MOVE', 'WORKER_8.*m3.*MOUNTAIN')
perform_action_matching('PRODUCE','TUNDRA')

# Perform secondary action to gain more coins

In [13]:
def perform_seconday_action_if_available():
    available_actions = client.get_available_actions()
    for action in ['ENLIST', 'DEPLOY', 'BUILD']:
        if action in available_actions:
            print(action)
            number = len(client.get_available_options(action))
            client.perform_action(action, random.randint(0, number))
            
for _ in range(30):
    perform_action_matching('BOLSTER', '')
    perform_seconday_action_if_available()
    
    perform_action_matching('TRADE', 'Popularity')
    perform_seconday_action_if_available()
        
    perform_action_matching('MOVE', 'GainCoin')
    perform_seconday_action_if_available()

    perform_action_matching('PRODUCE', random.choice(['MOUNTAIN', 'FOREST', 'TUNDRA']))
    perform_seconday_action_if_available()
    perform_action_matching('TRADE', 'OIL')
    perform_seconday_action_if_available()
    
    print_star_count()

BUILD
Number of stars  3
Number of stars  3
DEPLOY
DEPLOY
Number of stars  3
DEPLOY
DEPLOY
DEPLOY
DEPLOY
Number of stars  4
DEPLOY
DEPLOY
DEPLOY
DEPLOY
BUILD
Number of stars  4
Number of stars  4
Number of stars  4
DEPLOY
DEPLOY
Number of stars  4
DEPLOY
DEPLOY
DEPLOY
DEPLOY
DEPLOY
Number of stars  4
DEPLOY
DEPLOY
DEPLOY
DEPLOY
Number of stars  4
DEPLOY
DEPLOY
DEPLOY
DEPLOY
BUILD
Number of stars  4
Number of stars  4
DEPLOY
Number of stars  4
DEPLOY
DEPLOY
DEPLOY
DEPLOY
DEPLOY
BUILD
Number of stars  5
Number of stars  5
DEPLOY
Number of stars  5
DEPLOY
DEPLOY
DEPLOY
DEPLOY
DEPLOY
BUILD
Number of stars  5
DEPLOY
BUILD
BUILD
DEPLOY
DEPLOY
BUILD
Number of stars  5
DEPLOY
BUILD
BUILD
DEPLOY
DEPLOY
BUILD
Number of stars  5
DEPLOY
BUILD
BUILD
DEPLOY
DEPLOY
BUILD
Number of stars  5
DEPLOY
BUILD
BUILD
DEPLOY
DEPLOY
BUILD
Number of stars  5
DEPLOY
BUILD
BUILD
DEPLOY
DEPLOY
BUILD
Number of stars  5
DEPLOY
BUILD
BUILD
DEPLOY
DEPLOY
BUILD
Number of stars  5
DEPLOY
BUILD
BUILD
DEPLOY
DEPLOY
BUILD
N